# Introducción a Dash

Dash es un framework de Python para crear aplicaciones web. Está escrito sobre Flask, Plotly.js y React.js, ideada para los python-lovers y suficiente para construir aplicaciones web interactivas. 


Veamos una charla de Plotcon en la que se explica la motivación y los beneficios que trae Dash. 

In [2]:
from IPython.display import IFrame
IFrame(width="560",height="315", src="https://www.youtube.com/embed/5BAthiN0htc")

## Instalación y definición de Layout
En consola escriba:

```
$ pip install dash
```


In [ ]:
!pip install dash

Las aplicaciones Dash se componen de dos partes. La primera parte es el "Layout" (diseño) de la aplicación y describe cómo se ve la aplicación. La segunda parte describe la interactividad de la aplicación. Dash proporciona clases de Python para todos los componentes visuales de la aplicación. Mantenemos un conjunto de componentes en la biblioteca dash_core_components y dash_html_components, pero también puede crear los suyos propios con JavaScript y React.js.


In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#app = dash.Dash(external_stylesheets=external_stylesheets)
app = dash.Dash(__name__)#armamos un dash


df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})#Base de datos a utilizar

#figura creada con plotly.express
fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

#diseño del app
app.layout = html.Div(children=[
    html.H1(children='Encabezado'),

    html.Div(children='''
        Mi primera app con Dash, un as bajo la manga de la visulaización de datos.
    '''),

    dcc.Graph(
        figure=fig
    )
])

In [ ]:
app.run_server(port="1121")

También tenemos a la mano jupyter-dash, diseñado espacialmente para el trabajo con cuadernos de Jupyter.

In [5]:
from jupyter_dash import JupyterDash
app2=JupyterDash(__name__)
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})#Base de datos a utilizar

#figura creada con plotly.express
fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

#diseño del app
app2.layout = html.Div(children=[
    html.H1(children='Encabezado'),

    html.Div(children='''
        Mi primera app con Dash, un as bajo la manga de la visulaización de datos.
    '''),

    dcc.Graph(
        figure=fig
    )
])

In [6]:
app2.run_server(mode="inline")

## Más implementaciones (Callbacks)

Atrás se vio que con app.layout se describe cómo se ve la aplicación usando un árbol jerárquico de componentes. La biblioteca dash_html_components proporciona clases para todas las etiquetas HTML, y los argumentos de las palabras clave describen los atributos HTML como style, className e id. La biblioteca dash_core_components genera componentes de nivel superior como controles y gráficos.

Este capítulo describe cómo crear sus aplicaciones Dash usando funciones de devolución de llamada: Funciones de Python que son llamadas automáticamente por Dash cada vez que cambia la propiedad de un componente de entrada.

Comencemos con un ejemplo simple de una aplicación Dash interactiva.

In [7]:
import dash
from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__,external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H6("¡Cambie el valor en el cuadro de texto para ver los callback en acción!"),
    html.Div(["Input: ",
              dcc.Input(id='my-input', value='initial value', type='text')]),
    html.Br(),
    html.Div(id='my-output'),

])
@app.callback(
    Output(component_id='my-output', component_property='children'),
    [Input(component_id='my-input', component_property='value')]
)
def actualiza(input_value):
    return 'Output: {}'.format(input_value)

In [8]:
app.run_server(debug=True,port="11211")

Dash app running on http://127.0.0.1:11211/


Analicemos este ejemplo:

1. Las "entradas" y "salidas" de la interfaz de nuestra aplicación se describen declarativamente como los argumentos del decorador @app.callback.

       1. Al escribir este decorador, le estamos diciendo a Dash que llame a esta función por nosotros siempre que el valor del componente "entrada" (el cuadro de texto) cambie para actualizar los hijos del componente "salida" en la página (el div HTML ).

        2. Puede usar cualquier nombre para la función dentro del decorador @app.callback. La convención es que el nombre describe la(s) salida(s) de devolución de llamada.

        3. Puede usar cualquier nombre para los argumentos de la función, pero debe usar los mismos nombres dentro de la función de devolución de llamada que usa en su definición, al igual que en una función Python normal. Los argumentos son posicionales: primero los elementos de entrada y luego los elementos de estado se dan en el mismo orden que en el decorador.

        4. Debe usar la misma identificación que le dio a un componente Dash en el app.layout cuando se refiere a él como una entrada o salida del decorador @app.callback.

        5. El decorador @app.callback debe estar directamente encima de la declaración del mismo. Si hay una línea en blanco entre el decorador y la definición de la función, el registro de devolución de llamada no se realizará correctamente.

        6. Si tiene curiosidad acerca de lo que significa la sintaxis del decorador bajo el capó, puede leer esta respuesta de StackOverflow y aprender más sobre decoradores leyendo PEP 318 - Decoradores para funciones y métodos.
        
        

2. En Dash, las entradas y salidas de nuestra aplicación son simplemente las propiedades de un componente en particular. En este ejemplo, nuestra entrada es la propiedad "value" del componente que tiene el ID "my-input". Nuestra salida es la propiedad "children" del componente con el ID "my-output".
3. Siempre que cambie una propiedad de entrada, la función que envuelve el decorador se llamará automáticamente. Dash proporciona a la función el nuevo valor de la propiedad de entrada como argumento de entrada y Dash actualiza la propiedad del componente de salida con lo que devolvió la función.
 4. Los keywords  component_id y component_property son opcionales (solo hay dos argumentos para cada uno de esos objetos). Se incluyen en este ejemplo para mayor claridad, pero se omitirán en el resto de la documentación por motivos de brevedad y legibilidad.
5. No confunda el objeto dash.dependencies.Input y el objeto dash_core_components.Input. El primero solo se usa en callbacks mientras el último es un componente real.
6. Observe que, cómo no establecemos un valor para la propiedad secundaria del componente my-output en el diseño. Cuando se inicia la aplicación Dash, automáticamente llama a todas las devoluciones de llamada con los valores iniciales de los componentes de entrada para completar el estado inicial de los componentes de salida. En este ejemplo, si especificaste algo como html.Div (id = 'my-output', children = 'Hello world'), se sobrescribirá cuando se inicie la aplicación.



In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        id='year-slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'),
    [Input('year-slider', 'value')])
def actualiza(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp", 
                     size="pop", color="continent", hover_name="country", 
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)

    return fig


In [10]:
app.run_server(debug=True,port="11233")

Dash app running on http://127.0.0.1:11233/


In [11]:
df

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,1987,9216418.0,Africa,62.351,706.157306
1700,Zimbabwe,1992,10704340.0,Africa,60.377,693.420786
1701,Zimbabwe,1997,11404948.0,Africa,46.809,792.449960
1702,Zimbabwe,2002,11926563.0,Africa,39.989,672.038623


En este ejemplo, la propiedad "value" del control deslizante es la entrada de la aplicación y la salida de la aplicación es la propiedad "figure" del gráfico. Siempre que cambia el valor del slider, Dash llama a la función de devolución de llamada `actualiza` con el nuevo valor. 

In [22]:
df

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [25]:
dff = df[df['Year'] == 2007]
dff

,Country Name,Indicator Name,Year,Value
33264,Arab World,"Agriculture, value added (% of GDP)",2007,7.105067
33265,Arab World,CO2 emissions (metric tons per capita),2007,4.136145
33266,Arab World,Domestic credit provided by financial sector (...,2007,37.337545
33267,Arab World,Electric power consumption (kWh per capita),2007,1992.421645
33268,Arab World,Energy use (kg of oil equivalent per capita),2007,1578.880692
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


In [31]:
x=dff[dff['Indicator Name'] == "Industry, value added (% of GDP)"]
x

,Country Name,Indicator Name,Year,Value
33273,Arab World,"Industry, value added (% of GDP)",2007,53.985710
33287,Caribbean small states,"Industry, value added (% of GDP)",2007,36.376771
33301,Central Europe and the Baltics,"Industry, value added (% of GDP)",2007,34.060113
33315,Early-demographic dividend,"Industry, value added (% of GDP)",2007,38.354118
33329,East Asia & Pacific,"Industry, value added (% of GDP)",2007,36.612933
...,...,...,...,...
36899,Virgin Islands,"Industry, value added (% of GDP)",2007,NaN
36913,West Bank and Gaza,"Industry, value added (% of GDP)",2007,23.234322
36927,"Yemen, Rep.","Industry, value added (% of GDP)",2007,48.730554
36941,Zambia,"Industry, value added (% of GDP)",2007,34.885422


In [34]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

available_indicators = df['Indicator Name'].unique()

app.layout = html.Div([
    html.H1("Un Dashboard más interesante"),
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    )
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value'),
     Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
                     y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
                     hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    fig.update_xaxes(title=xaxis_column_name, 
                     type='linear' if xaxis_type == 'Linear' else 'log') 

    fig.update_yaxes(title=yaxis_column_name, 
                     type='linear' if yaxis_type == 'Linear' else 'log') 

    return fig

In [35]:
app.run_server(debug=True,mode="inline")

Ahora hagamoslo para múltiples salidas:

In [19]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1("Un montón de información de un número"),
    dcc.Input(
        id='num-multi',
        type='number',
        value=5
    ),
    html.Table([
        html.Tr([html.Td(['x', html.Sup(2)]), html.Td(id='square')]),
        html.Tr([html.Td(['x', html.Sup(3)]), html.Td(id='cube')]),
        html.Tr([html.Td([2, html.Sup('x')]), html.Td(id='twos')]),
        html.Tr([html.Td([3, html.Sup('x')]), html.Td(id='threes')]),
        html.Tr([html.Td(['x', html.Sup('x')]), html.Td(id='x^x')]),
    ]),
])


@app.callback(
    [Output('square', 'children'),
     Output('cube', 'children'),
     Output('twos', 'children'),
     Output('threes', 'children'),
     Output('x^x', 'children')],
    [Input('num-multi', 'value')])
def callback_a(x):
    return x**2, x**3, 2**x, 3**x, x**x



In [21]:
app.run_server(debug=True,mode="inline")